Uniquely Staffed Consultants


In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, IntegerType, StringType

# Initialize Spark session

# 1. employees DataFrame
employees_schema = StructType([
    StructField("employee_id", IntegerType(), True),
    StructField("engagement_id", IntegerType(), True)
])

employees_data = [
    (1001, 1),
    (1001, 2),
    (1002, 1),
    (1003, 3),
    (1004, 4)
]

employees_df = spark.createDataFrame(employees_data, schema=employees_schema)

# 2. consulting_engagements DataFrame
engagements_schema = StructType([
    StructField("engagement_id", IntegerType(), True),
    StructField("project_name", StringType(), True),
    StructField("client_name", StringType(), True)
])

engagements_data = [
    (1, "SAP Logistics Modernization", "Department of Defense"),
    (2, "Oracle Cloud Migration", "Department of Education"),
    (3, "Trust & Safety Operations", "Google"),
    (4, "SAP IoT Cloud Integration", "Google")
]

consulting_engagements_df = spark.createDataFrame(engagements_data, schema=engagements_schema)

# Show the DataFrames
print("Employees DataFrame:")
employees_df.show(truncate=False)

print("Consulting Engagements DataFrame:")
consulting_engagements_df.show(truncate=False)


In [0]:
from pyspark.sql.functions import countDistinct

# Join employees with engagements to get client_name per employee
joined_df = employees_df.join(
    consulting_engagements_df,
    "engagement_id",
    "inner"
)

# Find the number of unique clients each employee is staffed to
employee_client_counts = joined_df.groupBy("employee_id").agg(
    countDistinct("client_name").alias("client_count"),
    # For later join, get the client_name(s) for each employee
    # If an employee is staffed to only one client, this will be that client
    # Otherwise, will be multiple rows per employee
)

# Filter employees exclusively staffed to a single client
exclusive_employees = joined_df.join(
    employee_client_counts,
    "employee_id"
).filter(
    "client_count = 1"
)

# For each client, count the number of exclusively staffed consultants
exclusive_counts = exclusive_employees.groupBy("client_name").agg(
    countDistinct("employee_id").alias("exclusively_staffed")
)

# For each client, count the total number of unique consultants staffed
total_counts = joined_df.groupBy("client_name").agg(
    countDistinct("employee_id").alias("total_staffed")
)

# Join both counts on client_name
result_df = total_counts.join(
    exclusive_counts,
    "client_name",
    "left"
).orderBy("client_name")

display(result_df)